# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

os.environ['AZURE_OPENAI_API_KEY'] = os.getenv("AZURE_OPENAI_CHAT_API_KEY")
os.environ['AZURE_OPENAI_ENDPOINT'] = os.getenv("AZURE_OPENAI_CHAT_ENDPOINT")
os.environ['OPENAI_API_VERSION'] = os.getenv("AZURE_OPENAI_CHAT_VERSION")
os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")

'gpt-35-turbo'

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

## Create our QandA application

In [2]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch

from langchain_openai import AzureChatOpenAI
from langchain.evaluation.qa import QAGenerateChain
from langchain_openai import AzureOpenAIEmbeddings

import langchain

from langchain.evaluation.qa import QAEvalChain

In [3]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [4]:
embeddings = AzureOpenAIEmbeddings(
        azure_deployment=os.getenv('AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'),
        azure_endpoint=os.getenv('AZURE_OPENAI_EMBEDDINGS_ENDPOINT'),
        api_key=os.getenv('AZURE_OPENAI_EMBEDDINGS_API_KEY'),
        api_version=os.getenv('AZURE_OPENAI_EMBEDDINGS_VERSION')
    )

In [5]:
db = DocArrayInMemorySearch.from_documents(
    data, 
    embeddings
)

/opt/anaconda3/envs/hackathon_venv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}

In [ ]:
retriever = db.as_retriever()

In [ ]:
llm = AzureChatOpenAI(azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
                azure_endpoint=os.getenv("AZURE_OPENAI_CHAT_ENDPOINT"),
                openai_api_key=os.getenv("AZURE_OPENAI_CHAT_API_KEY"),
                api_version=os.getenv("AZURE_OPENAI_CHAT_VERSION"),
                temperature=0.0)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [ ]:
data[10]

In [ ]:
data[11]

### Hard-coded examples

In [ ]:
examples = [{
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [ ]:
llm = AzureChatOpenAI(azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
                azure_endpoint=os.getenv("AZURE_OPENAI_CHAT_ENDPOINT"),
                openai_api_key=os.getenv("AZURE_OPENAI_CHAT_API_KEY"),
                api_version=os.getenv("AZURE_OPENAI_CHAT_VERSION"),
                temperature=0.0)

In [ ]:
example_gen_chain = QAGenerateChain.from_llm(llm)

In [ ]:
# the warning below can be safely ignored

In [ ]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)
new_examples = [i['qa_pairs'] for i in new_examples]

In [ ]:
data[0]

### Combine examples

In [ ]:
examples += new_examples

In [ ]:
qa.run(examples[0]["query"])

## Manual Evaluation

In [ ]:
langchain.debug = True

In [ ]:
qa.run(examples[0]["query"])

In [ ]:
# Turn off the debug mode
langchain.debug = False

## LLM assisted evaluation

In [ ]:
predictions = qa.apply(examples)

In [ ]:
llm = AzureChatOpenAI(azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
                azure_endpoint=os.getenv("AZURE_OPENAI_CHAT_ENDPOINT"),
                openai_api_key=os.getenv("AZURE_OPENAI_CHAT_API_KEY"),
                api_version=os.getenv("AZURE_OPENAI_CHAT_VERSION"),
                temperature=0.0)

In [ ]:
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [ ]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

## LangChain evaluation platform

The LangChain evaluation platform, LangChain Plus, can be accessed here https://www.langchain.plus/.  
Use the invite code `lang_learners_2023`

Reminder: Download your notebook to you local computer to save your work.